# Trabajo Computacional N°1: Implementación del juego inteligente *ATRAPADOS* en Python

## Curso "Inteligencia Artificial" - Seccion 1
### Profesor: Rolando A. Maguiña Pérez

## Estdiante: Sergio M. Valdivia Marcelo
### Mayo 2023

## Introducción

En el presente cuaderno computacional se presentara el codigo de la implementacion del juego **ATRAPADOS** en lenguaje Python.
Se explicara el funcionamiento de las funciones y variables establecidas, asi como los resultados del mismo.
Para más información sobre las caracteristicas del juego y su desarrollo como juego inteligente, se recomienda analisis el informe adjuntado con el cuaderno computacional, o acceder al siguiente enlace a repositio GitHUb

## Inicialización del juego

Los estados del juego se basan en el tablero. El cual se representa como una **lista**  de 32 elementos correspondientes a las casillas del tablero. 
Se le asigna al `jugadorUno` el numero 1 y al `jugadorDos` el numero 2. Estos valores seran la referencia en la lista para representar las fichas de cada jugador. Si un elemento de la lista es 1, sera una ficha del jugador uno (X), si es 2 sera una fichas para el jugador 2 (Y), y si el elemnto es 0, se considera una casilla vacia.

In [2]:
import random #Se utilizara luego en las estrategias de juego

jugadorUno = 1
jugadorDos = 2

A continuación se presenta la funciones para la creación y representación del tablero:

`crearTablero()`: Genera el tablero en el estado inicial del juego.

`imprimirTablero()`: Imprime el tablero segun su contenido. Utiliza la función `convertirALetras()`, que retorna una letra o espacio en blanco segun el valor en el tablero que se envia.

`imprimirReferencia()`: imprime el tablero con las posiciones correspondientes de cada casilla a modod e referencia de ubicacion.

In [3]:
def crearTablero():
    tablero = []
    for i in range(0,32):
        if i%8 == 0:
            tablero.append(1)
        elif i%8-8 == -1 :
            tablero.append(2)
        else:
            tablero.append(0)
    return tablero

In [4]:
def convertirALetras(L):
    if (L == jugadorUno): return 'O'
    elif (L == jugadorDos): return 'X'
    elif (L == 0): return " "
    

def imprimirTablero(tablero):
    print("""\n
                    ------ TABLERO ------
            """)
    for i in range(0,4):
        print('-----------------------------------------------------------------')
        for e in range(0,7):
            print("|   {0}  ".format(convertirALetras(tablero[(8*i)+e])),end=" ")
        print("|   {0}   |".format(convertirALetras(tablero[(8*i)+e+1])))
    print('-----------------------------------------------------------------')    
    print("\n")

In [5]:
def imprimirReferencia():
    print("\n")
    for i in range(0,4):
        print('-----------------------------------------------------------------')
        for e in range(0,8):
            val=(8*i)+e+1
            if val < 10: posiRef=" "+str(val)
            else: posiRef=str(val)
            if e == 7:
                print("|  {0}   |".format(posiRef))
            else:
                print("|  {0}  ".format(posiRef),end=" ")
    print('-----------------------------------------------------------------')

## Obtención de espacios disponibles

Teniendo en cuenta las reglas del juego, se sabe que las fichas solo se pueden mover a ciertas casillas. De tal manera que es necesario obtener la **casillas disponibles y validas** para poder hacer los movimientos. Asu vez, como se vera mas adelante, tambien resulta necesario el saber la **cantidad total** de posiciones validas. Para esto se tiene las siguientes funciones:

- `espaciosEnFilaDisponibles()`: obtiene las casillas disponibles para   moverse de determinada fichas y jugador. Retornar una **lista con las posiciones** referenciales en orden ascendente.

- `calculaTotalEspaciosDisponibles()`: utiliza las funcion `espaciosEnFilaDisponibles()` con todas las fichas del jugador, de tal manera que va acumulando la **cantidad de posiciones** disponibles para cada ficha, y retorna el valor obtenido. 

In [6]:
def espaciosEnFilaDisponibles(tablero,ficha,jugador):
    espacios = []
    inicio = (ficha-1)*8
    fin = (ficha-1)*8+7
    
    if jugador == jugadorUno:
        while (inicio < fin+1):
            if tablero[inicio] == jugadorDos:
                break
            if tablero[inicio] != jugadorUno:
                espacios.append(inicio)
            inicio += 1
    if jugador == jugadorDos:
        while (fin > inicio-1):
            if tablero[fin] == jugadorUno:
                break
            if tablero[fin] != jugadorDos:
                espacios.append(fin)
            fin -= 1
        espacios.reverse()
        
    return espacios

In [7]:
def calculaTotalEspaciosDisponibles(tablero,jugador):
    suma = 0
    for i in range(1,5):
        disponiblesTotal = espaciosEnFilaDisponibles(tablero,i,jugador)
        suma += len(disponiblesTotal)
    return suma

## Posicion de la ficha

Las siguientes funciones ayudan a obtener información sobre la referencia a una fichas o su posición.

- `buscarPosiActualFicha()`: retorna la posicion actual de la ficha elegida. Busca en el tablero por filas, hasta que encuentre la ficha. 

- `buscarFicha()`: retorna la ficha correspondiente a una poscion del tablero. Al dividirce por filas, una ficha siempre esta en un rango de valores de una fila.

In [8]:
def buscarPosiActualFicha(tablero, ficha, jugador):
    posiActu = -1
    for i in range(0,8):
        if tablero[((ficha-1)*8)+i] == jugador:
            posiActu = ((ficha-1)*8)+i
            break
    return posiActu

In [9]:
def buscarFicha(posi):
    if posi >=0 and posi <=7:
        return 1
    if posi >=8 and posi <=15:
        return 2
    if posi >=16 and posi <=23:
        return 3
    if posi >=24 and posi <=31:
        return 4

## Validación de ganadores

Durante el desarrollo del juego, es neceasario validar si el estado actual del tablero representa la victoria de algun jugador.

Existen 2 tipos de funciones de verificacion: La primera verifica en estado **actual** del tablero para encontrar un ganador, la segunda verifica un estado **fututo** para ver si es un estado en donde gana algun jugador.

- `ganadorPartida()`: Recibe el tablero parar verificar si existe algun ganador, retornar verdadero o falso. Para esto, utiliza la funcion `calculaTotalEspaciosDisponibles()` para obtener la cantidad de posibles jugadas que le queda a un jugador. Si en algun caso, un jugador tiene 24 espacios para jugar y su oponente tiene 0, entonces se considera que existe un jugador y retorna verdadero, sino retorna falso y continua el juego. Esta condición no es una funcion de evaluación, solo es la verificacion de un estado final.

- `ganadorJ1()` y `ganadorJ2()`: Son funciones especificas para cada jugado y verificar si dicho gana la partida. Pese a que utilizan la misma evaluación, la diferencia con la función `ganadorPartida()` es que estas son para estados futuros (sucesores), y verifican la victoria para cada jugador, y no de forma general.

In [10]:
def ganadorPartida(tablero):
    disponiblesJ1 = calculaTotalEspaciosDisponibles(tablero,jugadorUno)
    disponiblesJ2 = calculaTotalEspaciosDisponibles(tablero,jugadorDos)
    
    ganaJ1 = False
    ganaJ2 = False
    
    if disponiblesJ1 == 24 and disponiblesJ2 == 0:
        ganaJ1 = True
    if disponiblesJ1 == 0 and disponiblesJ2 == 24:
        ganaJ2 = True
            
    if ganaJ1 == True and ganaJ2 == False:
        ganador = ganaJ1
    elif ganaJ1 == False and ganaJ2 == True:
        ganador = ganaJ2
    else:
        ganador = False
    return ganador

In [11]:
def ganadorJ1(tablero):
    disponiblesJ1 = calculaTotalEspaciosDisponibles(tablero,jugadorUno) 
    disponiblesJ2 = calculaTotalEspaciosDisponibles(tablero,jugadorDos)
    if disponiblesJ1 == 24 and disponiblesJ2 == 0:
        return True
    else:
        return False

In [12]:
def ganadorJ2(tablero):
    disponiblesJ1 = calculaTotalEspaciosDisponibles(tablero,jugadorUno) 
    disponiblesJ2 = calculaTotalEspaciosDisponibles(tablero,jugadorDos)
    if disponiblesJ2 == 24 and disponiblesJ1 == 0:
        return True
    else:
        return False

## Movimiento

### Validad movimiento

Esta funcion de usa para valiar los movimientos ingresados por los juadores humanos. Se repite su ejecucion hasta obtenet un resultado valido.
- `validarMovimiento()`: Primero le pide al ususario indicar la ficha que quiere jugar, y repite estos hasta ingresar una opcion valida. Luego le pide al usuario ingresar la posicion a la cual quiere mover la ficha. Mediante la funcion `espaciosEnFilaDisponibles()` valida si el movimiento esta permitido o no, y repite esto hasta el ingreso de un movimiento valido.

In [13]:
def validarMovimiento(tablero,jugador):
    vale=False
    
    while(not vale):
        ficha = int(input("\nIngrese la numero de ficha que quiere mover: "))
        if(ficha not in range(1,5)):
            print("Eleccion no valida!")
        else : vale = True
    vale = False
    
    movimientosPermitidos = espaciosEnFilaDisponibles(tablero,ficha,jugador)
    
    while(not vale):
        pos = int(input("\nIngrese su movimiento: "))
        if((pos-1) not in movimientosPermitidos):
            print("Movimiento no valido!")
        else : 
            pos = pos
            vale = True
    
    return pos

### Realizar movimiento

Las siguientes funcion sirven para realizar un movimiento en el tablero que previamente fue verificado. Estas funciones se utilzan mientras el juego esta en progreso, y tambien para la obtención de sucesores.
- `efectuarMovimiento()`: Obtieen el tablero, una posicion inicial, una posicion final, y el jugador correspondiente. Con esta información, busca la posición actual de la ficha elegida, reliza una copia del tablero y luego intercambia las posiciones para hacer que la ficha salga de su origen y se ponga en la nueva posición. Finalmente retorna el nuevo tablero con los cambios hechos.
- `sacarTurno()`: Se utiliza mas adelante para obtener a que jugador le toca turno para jugar.

In [14]:
def efectuarMovimiento(tablero, pos, ini, jugador):
    tableroCopia = tablero.copy()
    
    posiFichaActual = buscarPosiActualFicha(tablero,ini,jugador)

    tableroCopia[posiFichaActual] = 0
    tableroCopia[pos-1] = jugador
    
    return tableroCopia

In [15]:
def sacarTurno(jugador):
    if(jugador == jugadorUno):
        return jugadorDos
    if(jugador == jugadorDos):
        return jugadorUno

## Función de Evaluación

La función `funcionEvaluacion()` sirve para poder darle un peso a un estado, el cual sera utilzado como valor de evaluacion por alguna de las estrategias planteadas.
Como se plantea en el informe, la funcion de evaluacion es la diferencia entre el total de espacios disponibles de un jugador y la cantidad de espacios disponbles del oponente.

- `funcionEvaluacion()`: recibe el tablero actual y el jugador que se desea evaluar. Primero verifica si el tablero corresponde a un estado ganador del jugador (retornando un valor cercano al infinito positivo) o del oponente (retornando un valor cercano al infinito negativo). Si no ocurre esto, el valor se calcula mediante la operacion planteada con anterioridad, retornando el resultado.

In [16]:
def funcionEvaluacion(tablero,jugador):
    if ganadorJ1(tablero):
        return 666 * (1 if jugador == jugadorUno else -1)
    elif ganadorJ2(tablero):
        return 666 * (-1 if jugador == jugadorUno else 1)
    else:
        valorCaso = calculaTotalEspaciosDisponibles(tablero,jugador)
        if jugador == jugadorUno:
             mejorVal = valorCaso - calculaTotalEspaciosDisponibles(tablero,jugadorDos)
        if jugador == jugadorDos:
            mejorVal = valorCaso - calculaTotalEspaciosDisponibles(tablero,jugadorUno)
        return mejorVal

## Estategias de Juego

La implementación del juego establece que existen dos Estrategias de juego: Estrategia Aleatorio y Estrategia Primero el Mejor. Los cuales vana determianr el movimiento que va a realizar la PC.

### Sucesores

Los sucesores son los posibles estados del juego a los que se puede acceder un jugados desde el estado actual. 
- `obtenerSucesores()`: recibe el jugador y el estado actual del tablero, obtiene por cada ficha los espacios disponibles y con estos generado nuevos tableros como nuevos estados, por ultimo adjunta estos nuevos estados (tableros) en una lista y los retorna.

In [17]:
def obtenerSucesores(tablero,jugador):
    posibilidades = []
    
    for i in range(1,5):
        disponibles = espaciosEnFilaDisponibles(tablero,i,jugador)
        
        if len(disponibles) != 0:
            for e in disponibles:
                sucesor = efectuarMovimiento(tablero, e+1, i, jugador)
                posibilidades.append(sucesor)
    return posibilidades

### Estategia Aleatorio

En esta estrategia se elije uno de los suscesores del estado actual aleatoriamente.
- `seleccionAleatorio()`: teniendo en cuenta el jugador y el tablero actual, genera los sucesores y elige uno al azar. Luego recorre el estado hasta encontrar el movimiento a realizar y luego retornarlo.
- `estrategiaAleatorio()`: consolida y retorna en una tupla el movimiento obtenido en `seleccionAleatorio()` con su tipo de estrategia

In [18]:
def seleccionAleatorio(tablero, jugador):
    sucesores = obtenerSucesores(tablero,jugador)
    
    if len(sucesores)>0:
        sucesorAleatorio = random.choice(sucesores)
        z = sucesorAleatorio
    
        i = 0
        while (i<32):
            if(z[i] != tablero[i]):
                if z[i] != 0:
                    break
            i = i + 1
        return i+1
    else:
        return 1

In [19]:
def estrategiaAleatorio(tablero, jugador):
    valor = seleccionAleatorio(tablero, jugador)
    tupla = valor,"Movimiento por Aleatorio"
    return tupla

### Estategia Primero el Mejor

En esta estrategia antes de elegir un estado, los sucesores pasan por la función de evaluación, obteniendo su peso correspondiente. Luego se ordenan de forma ascendente segun su valor de evaluacion. Una vez realizado esto, se elige el estado de mayor valor de evaluacion como el nuevo estado y se recorre hasta obtener el nuevo movimiento.
- `sucesoresEvaluados()`: recibe la lista de sucesores y los pasa por la funcion `funcionEvaluacion()` para obtener sus valores de evaluacion. Retorna una **lista de listas** con los estados con sus respectivos pesos.
- `seleccionPrimeroMejor()`: primero obtiene los sucesores del estado actual segun el jugador con la funcion `obtenerSucesores()`. Luego manda los sucesores a la funcion `sucesoresEvaluados()` para su evaluación. Despues de eso los ordena de forma ascendente segun su valor de evaluación. Con esta lista ordenada, se elige el de mayor peso y se recorre el estado hasta encontrar el movimiento a realizar y luego retornarlo.
- `estrategiaPrimeroMejor()`: consolida y retorna en una tupla el movimiento obtenido en `seleccionPrimeroMejor()` con su tipo de estrategia

In [20]:
def sucesoresEvaluados(sucesores, jugador):
    sucesoresValores = []
    
    for caso in sucesores:
        eva = funcionEvaluacion(caso,jugador) 
        lista = [caso,eva]
        sucesoresValores.append(lista)
    return sucesoresValores

In [21]:
def seleccionPrimeroMejor(tablero, jugador):
    sucesores = obtenerSucesores(tablero,jugador)
    
    sucesoresValuados = sucesoresEvaluados( sucesores, jugador)
    
    sucesoresOrdenados = sorted(sucesoresValuados, key=lambda x: x[1])

    if len(sucesoresOrdenados)>0:
        sucesoresOrdenados = sucesoresOrdenados.pop()
        
        z = sucesoresOrdenados[0]
    
        i = 0
        while (i<32):
            if(z[i] != tablero[i]):
                if z[i] != 0:
                    break
            i = i + 1
        return i+1
    else:
        return 1

In [22]:
def estrategiaPrimeroMejor(tablero, jugador):
    valor = seleccionPrimeroMejor(tablero, jugador)
    tupla = valor,"Movimiento por Primero el Mejor"
    return tupla

### Eleccion de la estrategia:

En esta parte se establece la estrategia que va seguir el jugador maquina, existen 2 estrategias: Estrategia Aleatorio y Estrategia Primero el Mejor
- `escogerEstrategia()`: Obteneie la seleccion de la estrategia y elige la estrategia a seguir

In [23]:
def escogerEstrategia(tablero,seleccion,jugador):
    mejorMovi = -1
    if seleccion == 1:
        mejorMovi=estrategiaAleatorio(tablero,jugador)
    if seleccion == 2:
        mejorMovi=estrategiaPrimeroMejor(tablero,jugador)
    return mejorMovi   

## Jugadores

Existen 2 jugadors: El jugador humano, quien va a ingresar el movimiento que desea realizar; y el jugador PC, que va a realizar movimientos segun las estrategia elegida.

### Jugador Humano

- `movimientoHumano()`: Esta funcion permite al jugador humano ingresar su movimiento. Primero muestra la referencia de las posiciones con al función `imprimirReferencia()`, despues llama a la funcion `efectuarMovimiento()` para que el jugador ingrese su movimietno correctamente, luego realiza el movimiento con la función `efectuarMovimiento()` e imprime el nuevo tablero con `imprimirTablero()`. Despues de todo esto, valida si este ultimo movimiento hace que el jugador gana la partida con la función `ganadorPartida()`. Si esto pasa, se declara ganador al jugador, si no, prosigue con el juego mandando a jugar denuevo a un jugador humano con la función `movimientoHumano()`, o a un jugador PC con la función `movimientoAutonomo()`. Además, si se alcanzan el limite de jugadas permitidas, se cancela el juego.

In [24]:
def movimientoHumano(tablero,jugador,giro):
    print("JUGADA NUMERO: ",giro)
    print("Ahora toca jugar a humano JUGADOR",jugador)
    imprimirReferencia()

    pos = validarMovimiento(tablero,jugador)
    actu = buscarFicha(pos-1)
    
    nuevoTableroJugado = efectuarMovimiento(tablero,pos,actu,jugador)
    
    print("Movimiento: ",pos)
    imprimirTablero(nuevoTableroJugado)
    
    if ganadorPartida(nuevoTableroJugado) == True:
        
        print("\n\n\t\t ------------------------------")
        print("\t\t ----- GANA EL JUGADOR {0} ------".format(str(jugador)))
        print("\t\t ------------------------------\n\n")   
        
        inicio()
    else:
        if giro == 100:
                print("\n\t---CANTIDAD DE JUGADAS AGOTADAS---\n")
                inicio()
        else:
            if tipoJuego == 1:
                movimientoAutonomo(nuevoTableroJugado,sacarTurno(jugador),giro+1)
            if tipoJuego == 2:
                movimientoHumano(nuevoTableroJugado,sacarTurno(jugador),giro+1)

### Jugador PC

- `movimientoAutono()`: Esta funcion realiza las jugadas del jugador PC. Como es autonomo, no necesita presentar la referencia de posiciones. Primero llama a la funcion `escogerEstrategia()` que obtiene el movimiento a realizar segun la estrategia elegida, luego realiza el movimiento con la función `efectuarMovimiento()` e imprime el nuevo tablero con `imprimirTablero()`. Despues de todo esto, valida si este ultimo movimiento hace que el jugador gana la partida con la función `ganadorPartida()`. Si esto pasa, se declara ganador al jugador, si no, prosigue con el juego mandando a jugar denuevo a un jugador humano con la función `movimientoHumano()`, o a un jugador PC con la función `movimientoAutonomo()`. Además, si se alcanzan el limite de jugadas permitidas, se cancela el juego. 

In [25]:
def movimientoAutonomo(tablero,jugador,giro):
    print("JUGADA NUMERO: ",giro)
    print("Ahora toca jugar a computadora JUGADOR",jugador)

    mejorMovimiento = escogerEstrategia(tablero,dificultad,jugador)
    
    pos = mejorMovimiento[0]
    actu = buscarFicha(pos-1)
    estrategia = mejorMovimiento[1]

    nuevoTableroJugado = (efectuarMovimiento(tablero,pos,actu,jugador)) 
    
    print("Movimiento: ",pos)
    print("Estrategia: ",estrategia)
    
    imprimirTablero(nuevoTableroJugado)
    
    if ganadorPartida(nuevoTableroJugado) == True:
        print("\n\n\t\t ------------------------------")
        print("\t\t ----- GANA EL JUGADOR {0} ------".format(str(jugador)))
        print("\t\t ------------------------------\n\n")   
        
        inicio()
    else:
        if giro == 100:
                print("\n\t---CANTIDAD DE JUGADAS AGOTADAS---\n")
                inicio()
        else:
            if tipoJuego == 1:
                movimientoHumano(nuevoTableroJugado,sacarTurno(jugador),giro+1)
            if tipoJuego == 3:
                movimientoAutonomo(nuevoTableroJugado,sacarTurno(jugador),giro+1)

## Inicio del juego

Finalmente, para comenzar a jugar, se hace uso de la función `inicio()`, en donde se van a establecer el modo de juego y la dificultad de juego, asi como el primer turno. 
- `inicio()`: Primero genera el tablero inicial con la función `crearTablero()` y muestra un menu de 4 opciones en donde se elije el modo de juego o salir del programa. Una vez establecido esto, se imprime la referencia de posiciones con `imprimirReferencia()` y el tablero inicial con `imprimirTablero()`. Si se eligió la opcion 2 (Humano vs Humano), se ira directo a ejecutar la funcion `movimientoHumano()`, sino se mostrará un nuevo menú en donde se eligirá la dificultad del juego. Si inicialmente se eligió la opcion 3 (PC vs PC), se ira directo a ejecutar la función `movimientoAutonomo()`, sino se mostrará un ultimo menú en donde se eligira si inicia el jugador humano, con la función `movimientoHumano()`, o el jugador PC, con la función `movimientoAutonomo()`

In [26]:
def inicio():
    tableroInicial=crearTablero()
    vale=False
    global tipoJuego
    global dificultad
    global loop
    
    loop=1
    
    print("""\n
        ------ JUEGO "ATRAPADOS" ------
        Elija su modo de juego:
        [1] HUMANO vs PC
        [2] HUMANO vs HUMANO
        [3] PC vs PC
        [4] Salir
    """)
    
    while(not vale):
        opc1 = int(input("\nIngrese su opcion: "))
        if(opc1 not in range(1,5)):
            print("Eleccion no valida!")
        else : vale = True
    vale = False

    
    if opc1 == 4:
        print("\n")
        print("SALIENDO ...")   
    
    else:
        
        imprimirReferencia()
        imprimirTablero(tableroInicial)
        
        if opc1 == 2:
            tipoJuego = opc1
            movimientoHumano(tableroInicial,jugadorUno,loop)
        
        else: 
            print("""\n
                Elija su dificultad:
                [1] Facil (Aleatoria)
                [2] Media (Primero Mejor)
                [3] Salir
            """)

            while(not vale):
                opc2 = int(input("\nIngrese su opcion: "))
                if(opc2 not in range(1,4)):
                    print("Eleccion no valida!")
                else : vale = True
            vale = False       

            if opc2 == 3:
                print("\n")
                print("SALIENDO ...")   
           
            else:
                if opc2 == 1:
                    dificultad = opc2
                elif opc2 == 2:
                    dificultad = opc2
                    
                
                if opc1 == 1:
                    tipoJuego = opc1
                    
                    print("""\n
                        Quiere comenzar el juego?
                        [1] Si
                        [2] No
                        [3] Salir
                    """)

                    while(not vale):
                        opc3 = int(input("\nIngrese su opcion: "))
                        if(opc3 not in range(1,4)):
                            print("Eleccion no valida!")
                        else : vale = True
                    vale = False

                    if opc3 == 3:
                        print("\n")
                        print("SALIENDO ...")   
                    
                    else:
                        if opc3 == 1:
                            movimientoHumano(tableroInicial,jugadorUno,loop)
                        elif opc3 == 2:
                            movimientoAutonomo(tableroInicial,jugadorUno,loop)
                
                if opc1 == 3:
                    tipoJuego = opc1
                    movimientoAutonomo(tableroInicial,jugadorUno,loop)            

## CODIGO SIN COMENTARIOS

In [27]:
import random

jugadorUno = 1
jugadorDos = 2

def crearTablero():
    tablero = []
    for i in range(0,32):
        if i%8 == 0:
            tablero.append(1)
        elif i%8-8 == -1 :
            tablero.append(2)
        else:
            tablero.append(0)
    return tablero

def convertirALetras(L):
    if (L == jugadorUno): return 'O'
    elif (L == jugadorDos): return 'X'
    elif (L == 0): return " "
    
def imprimirTablero(tablero):
    print("""\n
                    ------ TABLERO ------
            """)
    for i in range(0,4):
        print('-----------------------------------------------------------------')
        for e in range(0,7):
            print("|   {0}  ".format(convertirALetras(tablero[(8*i)+e])),end=" ")
        print("|   {0}   |".format(convertirALetras(tablero[(8*i)+e+1])))
    print('-----------------------------------------------------------------')    
    print("\n")

def imprimirReferencia():
    print("\n")
    for i in range(0,4):
        print('-----------------------------------------------------------------')
        for e in range(0,8):
            val=(8*i)+e+1
            if val < 10: posiRef=" "+str(val)
            else: posiRef=str(val)
            if e == 7:
                print("|  {0}   |".format(posiRef))
            else:
                print("|  {0}  ".format(posiRef),end=" ")
    print('-----------------------------------------------------------------')

def espaciosEnFilaDisponibles(tablero,ficha,jugador):
    espacios = []
    inicio = (ficha-1)*8
    fin = (ficha-1)*8+7
    if jugador == jugadorUno:
        while (inicio < fin+1):
            if tablero[inicio] == jugadorDos:
                break
            if tablero[inicio] != jugadorUno:
                espacios.append(inicio)
            inicio += 1
    if jugador == jugadorDos:
        while (fin > inicio-1):
            if tablero[fin] == jugadorUno:
                break
            if tablero[fin] != jugadorDos:
                espacios.append(fin)
            fin -= 1
        espacios.reverse()
    return espacios

def calculaTotalEspaciosDisponibles(tablero,jugador):
    suma = 0
    for i in range(1,5):
        disponiblesTotal = espaciosEnFilaDisponibles(tablero,i,jugador)
        suma += len(disponiblesTotal)
    return suma

def buscarPosiActualFicha(tablero, ficha, jugador):
    posiActu = -1
    for i in range(0,8):
        if tablero[((ficha-1)*8)+i] == jugador:
            posiActu = ((ficha-1)*8)+i
            break
    return posiActu

def buscarFicha(posi):
    if posi >=0 and posi <=7:
        return 1
    if posi >=8 and posi <=15:
        return 2
    if posi >=16 and posi <=23:
        return 3
    if posi >=24 and posi <=31:
        return 4

def ganadorPartida(tablero):
    disponiblesJ1 = calculaTotalEspaciosDisponibles(tablero,jugadorUno)
    disponiblesJ2 = calculaTotalEspaciosDisponibles(tablero,jugadorDos)
    ganaJ1 = False
    ganaJ2 = False
    if disponiblesJ1 == 24 and disponiblesJ2 == 0:
        ganaJ1 = True
    if disponiblesJ1 == 0 and disponiblesJ2 == 24:
        ganaJ2 = True    
    if ganaJ1 == True and ganaJ2 == False:
        ganador = ganaJ1
    elif ganaJ1 == False and ganaJ2 == True:
        ganador = ganaJ2
    else:
        ganador = False
    return ganador

def ganadorJ1(tablero):
    disponiblesJ1 = calculaTotalEspaciosDisponibles(tablero,jugadorUno) 
    disponiblesJ2 = calculaTotalEspaciosDisponibles(tablero,jugadorDos)
    if disponiblesJ1 == 24 and disponiblesJ2 == 0:
        return True
    else:
        return False

def ganadorJ2(tablero):
    disponiblesJ1 = calculaTotalEspaciosDisponibles(tablero,jugadorUno) 
    disponiblesJ2 = calculaTotalEspaciosDisponibles(tablero,jugadorDos)
    if disponiblesJ2 == 24 and disponiblesJ1 == 0:
        return True
    else:
        return False

def validarMovimiento(tablero,jugador):
    vale=False
    while(not vale):
        ficha = int(input("\nIngrese la numero de ficha que quiere mover: "))
        if(ficha not in range(1,5)):
            print("Eleccion no valida!")
        else : vale = True
    vale = False
    movimientosPermitidos = espaciosEnFilaDisponibles(tablero,ficha,jugador)
    while(not vale):
        pos = int(input("\nIngrese su movimiento: "))
        if((pos-1) not in movimientosPermitidos):
            print("Movimiento no valido!")
        else : 
            pos = pos
            vale = True
    return pos

def efectuarMovimiento(tablero, pos, ini, jugador):
    tableroCopia = tablero.copy()    
    posiFichaActual = buscarPosiActualFicha(tablero,ini,jugador)
    tableroCopia[posiFichaActual] = 0
    tableroCopia[pos-1] = jugador    
    return tableroCopia

def sacarTurno(jugador):
    if(jugador == jugadorUno):
        return jugadorDos
    if(jugador == jugadorDos):
        return jugadorUno

def funcionEvaluacion(tablero,jugador):
    if ganadorJ1(tablero):
        return 666 * (1 if jugador == jugadorUno else -1)
    elif ganadorJ2(tablero):
        return 666 * (-1 if jugador == jugadorUno else 1)
    else:
        valorCaso = calculaTotalEspaciosDisponibles(tablero,jugador)
        if jugador == jugadorUno:
             mejorVal = valorCaso - calculaTotalEspaciosDisponibles(tablero,jugadorDos)
        if jugador == jugadorDos:
            mejorVal = valorCaso - calculaTotalEspaciosDisponibles(tablero,jugadorUno)
        return mejorVal

def obtenerSucesores(tablero,jugador):
    posibilidades = []    
    for i in range(1,5):
        disponibles = espaciosEnFilaDisponibles(tablero,i,jugador)        
        if len(disponibles) != 0:
            for e in disponibles:
                sucesor = efectuarMovimiento(tablero, e+1, i, jugador)
                posibilidades.append(sucesor)
    return posibilidades

def seleccionAleatorio(tablero, jugador):
    sucesores = obtenerSucesores(tablero,jugador)    
    if len(sucesores)>0:
        sucesorAleatorio = random.choice(sucesores)
        z = sucesorAleatorio    
        i = 0
        while (i<32):
            if(z[i] != tablero[i]):
                if z[i] != 0:
                    break
            i = i + 1
        return i+1
    else:
        return 1

def estrategiaAleatorio(tablero, jugador):
    valor = seleccionAleatorio(tablero, jugador)
    tupla = valor,"Movimiento por Aleatorio"
    return tupla

def sucesoresEvaluados(sucesores, jugador):
    sucesoresValores = []
    for caso in sucesores:
        eva = funcionEvaluacion(caso,jugador) 
        lista = [caso,eva]
        sucesoresValores.append(lista)
    return sucesoresValores

def seleccionPrimeroMejor(tablero, jugador):
    sucesores = obtenerSucesores(tablero,jugador)    
    sucesoresValuados = sucesoresEvaluados( sucesores, jugador)    
    sucesoresOrdenados = sorted(sucesoresValuados, key=lambda x: x[1])
    if len(sucesoresOrdenados)>0:
        sucesoresOrdenados = sucesoresOrdenados.pop()        
        z = sucesoresOrdenados[0]    
        i = 0
        while (i<32):
            if(z[i] != tablero[i]):
                if z[i] != 0:
                    break
            i = i + 1
        return i+1
    else:
        return 1

def estrategiaPrimeroMejor(tablero, jugador):
    valor = seleccionPrimeroMejor(tablero, jugador)
    tupla = valor,"Movimiento por Primero el Mejor"
    return tupla

def escogerEstrategia(tablero,seleccion,jugador):
    mejorMovi = -1
    if seleccion == 1:
        mejorMovi=estrategiaAleatorio(tablero,jugador)
    if seleccion == 2:
        mejorMovi=estrategiaPrimeroMejor(tablero,jugador)
    return mejorMovi  

def movimientoHumano(tablero,jugador,giro):
    print("JUGADA NUMERO: ",giro)
    print("Ahora toca jugar a humano JUGADOR",jugador)
    imprimirReferencia()
    pos = validarMovimiento(tablero,jugador)
    actu = buscarFicha(pos-1)    
    nuevoTableroJugado = efectuarMovimiento(tablero,pos,actu,jugador)    
    print("Movimiento: ",pos)
    imprimirTablero(nuevoTableroJugado)    
    if ganadorPartida(nuevoTableroJugado) == True:        
        print("\n\n\t\t ------------------------------")
        print("\t\t ----- GANA EL JUGADOR {0} ------".format(str(jugador)))
        print("\t\t ------------------------------\n\n")           
        inicio()
    else:
        if giro == 100:
                print("\n\t---CANTIDAD DE JUGADAS AGOTADAS---\n")
                inicio()
        else:
            if tipoJuego == 1:
                movimientoAutonomo(nuevoTableroJugado,sacarTurno(jugador),giro+1)
            if tipoJuego == 2:
                movimientoHumano(nuevoTableroJugado,sacarTurno(jugador),giro+1)

def movimientoAutonomo(tablero,jugador,giro):
    print("JUGADA NUMERO: ",giro)
    print("Ahora toca jugar a computadora JUGADOR",jugador)
    mejorMovimiento = escogerEstrategia(tablero,dificultad,jugador)    
    pos = mejorMovimiento[0]
    actu = buscarFicha(pos-1)
    estrategia = mejorMovimiento[1]
    nuevoTableroJugado = (efectuarMovimiento(tablero,pos,actu,jugador))     
    print("Movimiento: ",pos)
    print("Estrategia: ",estrategia)    
    imprimirTablero(nuevoTableroJugado)    
    if ganadorPartida(nuevoTableroJugado) == True:
        print("\n\n\t\t ------------------------------")
        print("\t\t ----- GANA EL JUGADOR {0} ------".format(str(jugador)))
        print("\t\t ------------------------------\n\n")           
        inicio()
    else:
        if giro == 100:
                print("\n\t---CANTIDAD DE JUGADAS AGOTADAS---\n")
                inicio()
        else:
            if tipoJuego == 1:
                movimientoHumano(nuevoTableroJugado,sacarTurno(jugador),giro+1)
            if tipoJuego == 3:
                movimientoAutonomo(nuevoTableroJugado,sacarTurno(jugador),giro+1)

def inicio():
    tableroInicial=crearTablero()
    vale=False
    global tipoJuego
    global dificultad
    global loop    
    loop=1    
    print("""\n
        ------ JUEGO "ATRAPADOS" ------
        Elija su modo de juego:
        [1] HUMANO vs PC
        [2] HUMANO vs HUMANO
        [3] PC vs PC
        [4] Salir
    """)    
    while(not vale):
        opc1 = int(input("\nIngrese su opcion: "))
        if(opc1 not in range(1,5)):
            print("Eleccion no valida!")
        else : vale = True
    vale = False   
    if opc1 == 4:
        print("\n")
        print("SALIENDO ...")     
    else:        
        imprimirReferencia()
        imprimirTablero(tableroInicial)        
        if opc1 == 2:
            tipoJuego = opc1
            movimientoHumano(tableroInicial,jugadorUno,loop)        
        else: 
            print("""\n
                Elija su dificultad:
                [1] Facil (Aleatoria)
                [2] Media (Primero Mejor)
                [3] Salir
            """)
            while(not vale):
                opc2 = int(input("\nIngrese su opcion: "))
                if(opc2 not in range(1,4)):
                    print("Eleccion no valida!")
                else : vale = True
            vale = False       
            if opc2 == 3:
                print("\n")
                print("SALIENDO ...")   
            else:
                if opc2 == 1:
                    dificultad = opc2
                elif opc2 == 2:
                    dificultad = opc2
                if opc1 == 1:
                    tipoJuego = opc1                    
                    print("""\n
                        Quiere comenzar el juego?
                        [1] Si
                        [2] No
                        [3] Salir
                    """)
                    while(not vale):
                        opc3 = int(input("\nIngrese su opcion: "))
                        if(opc3 not in range(1,4)):
                            print("Eleccion no valida!")
                        else : vale = True
                    vale = False
                    if opc3 == 3:
                        print("\n")
                        print("SALIENDO ...")   
                    else:
                        if opc3 == 1:
                            movimientoHumano(tableroInicial,jugadorUno,loop)
                        elif opc3 == 2:
                            movimientoAutonomo(tableroInicial,jugadorUno,loop)                
                if opc1 == 3:
                    tipoJuego = opc1
                    movimientoAutonomo(tableroInicial,jugadorUno,loop) ## Introducción

## Experimentos

### Experimento 1

In [28]:
inicio()



        ------ JUEGO "ATRAPADOS" ------
        Elija su modo de juego:
        [1] HUMANO vs PC
        [2] HUMANO vs HUMANO
        [3] PC vs PC
        [4] Salir
    

Ingrese su opcion: 1


-----------------------------------------------------------------
|   1   |   2   |   3   |   4   |   5   |   6   |   7   |   8   |
-----------------------------------------------------------------
|   9   |  10   |  11   |  12   |  13   |  14   |  15   |  16   |
-----------------------------------------------------------------
|  17   |  18   |  19   |  20   |  21   |  22   |  23   |  24   |
-----------------------------------------------------------------
|  25   |  26   |  27   |  28   |  29   |  30   |  31   |  32   |
-----------------------------------------------------------------


                    ------ TABLERO ------
            
-----------------------------------------------------------------
|   O   |       |       |       |       |       |       |   X   |
---------------------


Ingrese la numero de ficha que quiere mover: 4

Ingrese su movimiento: 26
Movimiento:  26


                    ------ TABLERO ------
            
-----------------------------------------------------------------
|       |       |       |       |       |       |   O   |   X   |
-----------------------------------------------------------------
|       |       |       |       |       |       |   O   |   X   |
-----------------------------------------------------------------
|   O   |   X   |       |       |       |       |       |       |
-----------------------------------------------------------------
|       |   O   |   X   |       |       |       |       |       |
-----------------------------------------------------------------


JUGADA NUMERO:  8
Ahora toca jugar a computadora JUGADOR 2
Movimiento:  28
Estrategia:  Movimiento por Primero el Mejor


                    ------ TABLERO ------
            
-----------------------------------------------------------------
|       |    


Ingrese la numero de ficha que quiere mover: 4

Ingrese su movimiento: 30
Movimiento:  30


                    ------ TABLERO ------
            
-----------------------------------------------------------------
|       |       |       |       |       |       |   O   |   X   |
-----------------------------------------------------------------
|       |       |       |       |       |       |   O   |   X   |
-----------------------------------------------------------------
|   O   |   X   |       |       |       |       |       |       |
-----------------------------------------------------------------
|       |       |       |       |       |   O   |   X   |       |
-----------------------------------------------------------------


JUGADA NUMERO:  16
Ahora toca jugar a computadora JUGADOR 2
Movimiento:  32
Estrategia:  Movimiento por Primero el Mejor


                    ------ TABLERO ------
            
-----------------------------------------------------------------
|       |   


Ingrese la numero de ficha que quiere mover: 3

Ingrese su movimiento: 20
Movimiento:  20


                    ------ TABLERO ------
            
-----------------------------------------------------------------
|       |       |       |       |       |       |   O   |   X   |
-----------------------------------------------------------------
|       |       |       |       |       |       |   O   |   X   |
-----------------------------------------------------------------
|       |       |       |   O   |   X   |       |       |       |
-----------------------------------------------------------------
|       |       |       |       |       |       |   O   |   X   |
-----------------------------------------------------------------


JUGADA NUMERO:  24
Ahora toca jugar a computadora JUGADOR 2
Movimiento:  22
Estrategia:  Movimiento por Primero el Mejor


                    ------ TABLERO ------
            
-----------------------------------------------------------------
|       |   

### Experimento 2

In [29]:
inicio()



        ------ JUEGO "ATRAPADOS" ------
        Elija su modo de juego:
        [1] HUMANO vs PC
        [2] HUMANO vs HUMANO
        [3] PC vs PC
        [4] Salir
    

Ingrese su opcion: 3


-----------------------------------------------------------------
|   1   |   2   |   3   |   4   |   5   |   6   |   7   |   8   |
-----------------------------------------------------------------
|   9   |  10   |  11   |  12   |  13   |  14   |  15   |  16   |
-----------------------------------------------------------------
|  17   |  18   |  19   |  20   |  21   |  22   |  23   |  24   |
-----------------------------------------------------------------
|  25   |  26   |  27   |  28   |  29   |  30   |  31   |  32   |
-----------------------------------------------------------------


                    ------ TABLERO ------
            
-----------------------------------------------------------------
|   O   |       |       |       |       |       |       |   X   |
---------------------


Ingrese su opcion: 4


SALIENDO ...


### Experimento 3

In [31]:
inicio()



        ------ JUEGO "ATRAPADOS" ------
        Elija su modo de juego:
        [1] HUMANO vs PC
        [2] HUMANO vs HUMANO
        [3] PC vs PC
        [4] Salir
    

Ingrese su opcion: 1


-----------------------------------------------------------------
|   1   |   2   |   3   |   4   |   5   |   6   |   7   |   8   |
-----------------------------------------------------------------
|   9   |  10   |  11   |  12   |  13   |  14   |  15   |  16   |
-----------------------------------------------------------------
|  17   |  18   |  19   |  20   |  21   |  22   |  23   |  24   |
-----------------------------------------------------------------
|  25   |  26   |  27   |  28   |  29   |  30   |  31   |  32   |
-----------------------------------------------------------------


                    ------ TABLERO ------
            
-----------------------------------------------------------------
|   O   |       |       |       |       |       |       |   X   |
---------------------


Ingrese la numero de ficha que quiere mover: 4

Ingrese su movimiento: 29
Movimiento:  29


                    ------ TABLERO ------
            
-----------------------------------------------------------------
|   O   |   X   |       |       |       |       |       |       |
-----------------------------------------------------------------
|   O   |   X   |       |       |       |       |       |       |
-----------------------------------------------------------------
|       |   O   |   X   |       |       |       |       |       |
-----------------------------------------------------------------
|       |       |       |   O   |   X   |       |       |       |
-----------------------------------------------------------------


JUGADA NUMERO:  9
Ahora toca jugar a computadora JUGADOR 1
Movimiento:  17
Estrategia:  Movimiento por Aleatorio


                    ------ TABLERO ------
            
-----------------------------------------------------------------
|   O   |   X   |   

### Experimento en clase

In [ ]:
inicio()